# 사주AI
* 만세력 해석을 기반으로 명언생성하는 인공지능 모델 개발

1)생년월일입력

2)만세력 도출(api)

3)해석문장 생성(gpt-한자 to 영어)

4)keywords, topic, summary 

5)키워드기반 문장생성(option)

6)명언DB - 해석한 만세력 결과 요약문장 문서유사도 계산하여 가장 적합한(유사한)명언 추출

In [1]:
# 생년월일입력

In [2]:
# 만세력 도출
# api를 사용하거나 제작해도 됨!

In [4]:
# 만세력 해석
# !pip install hanjaq

     |████████████████████████████████| 41 kB 2.4 MB/s eta 0:00:011
  Created wheel for hanjaq: filename=hanjaq-0.0.4-py3-none-any.whl size=39293 sha256=75b6f0fc8a860faeb386cc958c29ac366d0b9eea02d5220b9906babb4d5e9a0e
  Stored in directory: /Users/kimkwangil/Library/Caches/pip/wheels/9f/ec/25/3cc67804c27334759a0305317e6afe6858fc47e5a64c256c5a
Successfully built hanjaq


In [7]:
from hanjaq import get_hanja_information

In [8]:
get_hanja_information("癸")

(['계'], 'last of the 10 Heavenly Stems')

In [9]:
get_hanja_information("戊")

(['무'], '5th of the 10 Heavenly Stems')

In [11]:
get_hanja_information("庚")

(['경'], '7th of the 10 Heavenly Stems; evening star')

In [12]:
get_hanja_information("辰")

(['진', '신'], '5th of the 12 Earth’s Branches; star; dragon')

In [13]:
get_hanja_information("壬") 

(['임'], '9th of the 10 Heavenly Stems')

In [14]:
get_hanja_information("寅")

(['인'], '3d of the 12 Earth’s Branches; tiger')

In [15]:
get_hanja_information('丑')

(['축', '추'], '2d of the 12 Earth’s Branches; ox')

In [16]:
get_hanja_information('寅')

(['인'], '3d of the 12 Earth’s Branches; tiger')

In [17]:
get_hanja_information('戊')


(['무'], '5th of the 10 Heavenly Stems')

In [20]:
# 한자 합치기
hanGroup = ['癸','戊','丑','寅','壬','庚','戊']


In [22]:
# 의미 추출
mInfo = []
for i in hanGroup:
    getInfo = get_hanja_information(i) 
    mInfo.append(getInfo[1])
mInfo

['last of the 10 Heavenly Stems',
 '5th of the 10 Heavenly Stems',
 '2d of the 12 Earth’s Branches; ox',
 '3d of the 12 Earth’s Branches; tiger',
 '9th of the 10 Heavenly Stems',
 '7th of the 10 Heavenly Stems; evening star',
 '5th of the 10 Heavenly Stems']

In [24]:
getMeaning = " ".join(map(str, mInfo))
getMeaning

'last of the 10 Heavenly Stems 5th of the 10 Heavenly Stems 2d of the 12 Earth’s Branches; ox 3d of the 12 Earth’s Branches; tiger 9th of the 10 Heavenly Stems 7th of the 10 Heavenly Stems; evening star 5th of the 10 Heavenly Stems'

In [26]:
# 핵심키워드 추출
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([getMeaning])
candidates = count.get_feature_names()

/Users/kimkwangil/opt/anaconda3/envs/py38NLP/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([getMeaning])
candidate_embeddings = model.encode(candidates)

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [32]:
# 이렇게 키워드를 추출해 볼 수 있음, 것을 문자으로 생서이한다면... 
# 여기서는 임의로 키워드 적용해볼것임
keywords

['9th', '7th', 'tiger', '5th', '10']

In [45]:
#keywords_self = ['tiger' , 'moon', 'people', 'subordinate', 'house', 'public', 'success']

keywords_self = ['tiger' , 'moon', 'people']


In [46]:
#!pip install keytotext --upgrade
#!pip install typing-extensions

In [52]:
# 이건 오류때문에 안됨, 현재 수정불가
# from keytotext import pipeline

# #nlp = pipeline("k2t")
# #nlp = pipeline("k2t-base")
# nlp = pipeline("mrm8488/t5-base-finetuned-common_gen")
# nlp(keywords_self)

ImportError: cannot import name 'Self' from 'typing_extensions' (/Users/kimkwangil/opt/anaconda3/envs/py38NLP/lib/python3.8/site-packages/typing_extensions.py)

In [48]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("gagan3012/keytotext-small")

model = AutoModelWithLMHead.from_pretrained("gagan3012/keytotext-small")

In [54]:

output = model.generate("tiger", num_beams=8, do_sample=True,
                        min_length=10, max_length=64)


AttributeError: 'str' object has no attribute 'shape'

In [55]:
# 명언 가져오기
import json

In [60]:
with open("data/quotes.json", "r") as st_json:

    qt = json.load(st_json)

qt

[{'Quote': "Don't cry because it's over, smile because it happened.",
  'Author': 'Dr. Seuss',
  'Tags': ['attributed-no-source',
   'cry',
   'crying',
   'experience',
   'happiness',
   'joy',
   'life',
   'misattributed-dr-seuss',
   'optimism',
   'sadness',
   'smile',
   'smiling '],
  'Popularity': 0.15566615566615566,
  'Category': 'life'},
 {'Quote': "Don't cry because it's over, smile because it happened.",
  'Author': 'Dr. Seuss',
  'Tags': ['attributed-no-source',
   'cry',
   'crying',
   'experience',
   'happiness',
   'joy',
   'life',
   'misattributed-dr-seuss',
   'optimism',
   'sadness',
   'smile',
   'smiling '],
  'Popularity': 0.15566615566615566,
  'Category': 'happiness'},
 {'Quote': "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
  'Author': 'Marilyn Monroe',
  'Tags': ['attributed-no-source',
   'bes

In [77]:
type(qt)

list

In [68]:
# 명언 한개 가져오기
qt[0]['Quote']

"Don't cry because it's over, smile because it happened."

In [66]:
# 총 명언 수
len(qt)

48391

In [81]:
# 명언만 추출

quoteList = []
k = 0
for i in qt:
   
    quoteList.append(qt[k]['Quote'])
    k += 1
    
quoteList

["Don't cry because it's over, smile because it happened.",
 "Don't cry because it's over, smile because it happened.",
 "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
 "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
 "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
 'Be yourself; everyone else is already taken.',
 "Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.",
 "Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.",
 "Two thin

In [83]:
# 명언을 pickle로 저장 및 불러오기
import pickle

# save
with open('data/quoteData.pickle', 'wb') as f:
    pickle.dump(quoteList, f, pickle.HIGHEST_PROTOCOL)

# load
with open('data/quoteData.pickle', 'rb') as f:
    quoteList = pickle.load(f)

In [84]:
quoteList

["Don't cry because it's over, smile because it happened.",
 "Don't cry because it's over, smile because it happened.",
 "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
 "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
 "I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",
 'Be yourself; everyone else is already taken.',
 "Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.",
 "Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.",
 "Two thin

In [85]:
# 가장 유사한 명언 추출하기
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [86]:
# 비교문장.. 만약 이러한 문장이 만세력을 통해서 생성되었다면, 여기서 가장 유사한 의미의 명언을 찾아내기 
li_comp_sent = ["It sure is tiring to run with the herd. You may feel the desire to get away from the rat race at work today. Maybe it is the monotonous and stressful environment in office that is pushing you to your limits. But you may seek a new direction and lifestyle, one that is less taxing."]


In [ ]:
# 명언과 입력문장 유사도 계산 
simility_resut_list = []
for sent in quoteList:
    for sent_ in li_comp_sent:
        sen_input =[sent, sent_]
        #Encoding:
        sen_embeddings = model.encode(sen_input)
        #let's calculate cosine similarity for sentence 0:
        sim_re = cosine_similarity(
            [sen_embeddings[0]],
            sen_embeddings[1:]
        )

        # 문장 유사도 비교 결과 input_sent_A. vs. input_sent_B
        cal_sim = sim_re[0][0]
        simility_resut_list.append(cal_sim)

In [ ]:
simility_resut_list

In [ ]:
# 가장 유사한 명언 도출
max_value = max(simility_resut_list)

In [ ]:
# 끝!